# 다음 영화 리뷰 댓글로 영화 제목 예측하기
<hr>

In [1]:
# 데이터 불러오기
import pandas as pd

df = pd.read_csv('../data/daum_movie_review.csv')
df.head(5)

,review,rating,date,title
0,돈 들인건 티가 나지만 보는 내내 하품만,1,2018.10.29,인피니티 워
1,몰입할수밖에 없다. 어렵게 생각할 필요없다. 내가 전투에 참여한듯 손에 땀이남.,10,2018.10.26,인피니티 워
2,이전 작품에 비해 더 화려하고 스케일도 커졌지만.... 전국 맛집의 음식들을 한데 ...,8,2018.10.24,인피니티 워
3,이 정도면 볼만하다고 할 수 있음!,8,2018.10.22,인피니티 워
4,재미있다,10,2018.10.20,인피니티 워


In [2]:
print(len(df))

14725


In [3]:
df['title'].value_counts()
# '신과함께'와 '코코'의 개수 차이가 많이 남 -> 불균형 데이터셋(Imbalanced dataset)

신과함께      4947
택시운전사     2322
인피니티 워    2042
범죄도시      1939
곤지암       1547
라라랜드      1150
코코         778
Name: title, dtype: int64

In [6]:
# train/test split
from sklearn.model_selection import train_test_split

# 비율 지정하지 않으면 train:test = 75:25로 분할
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['title'], random_state=0)

print("Train size: ", len(X_train))
print("Test size: ", len(X_test))

Train size:  11043
Test size:  3682


In [8]:
# 형태소 분석
from konlpy.tag import Okt
t = Okt()

print(t.morphs(X_train[2]))   # 형태소
print(t.nouns(X_train[2]))    # 명사

['이전', '작품', '에', '비해', '더', '화려하고', '스케일', '도', '커졌지만', '....', '전국', '맛집', '의', '음식', '들', '을', '한데', '모은', '것', '까지는', '좋았으나', '이', '걸', '모두', '한', '그릇', '에', '섞어', '버린', '듯', '한', '느낌', '...', '그래도', '다음', '작품', '을', '기대하게', '만든다', '...']
['이전', '작품', '더', '스케일', '전국', '맛집', '음식', '것', '걸', '모두', '그릇', '듯', '느낌', '다음', '작품']


In [11]:
# tfidf로 특성 추출
from sklearn.feature_extraction.text import TfidfVectorizer

# t.nouns를 토크나이저로 지정
tfidf = TfidfVectorizer(tokenizer=t.nouns, max_features=2000, min_df=5, max_df=0.5)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [12]:
# 로지스틱 회귀분석
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=1000)  

lr.fit(X_train_tfidf, y_train)

print("Train score: {:.3f}".format(lr.score(X_train_tfidf, y_train)))
print("Test score: {:.3f}".format(lr.score(X_test_tfidf, y_test)))

Train score: 0.756
Test score: 0.694


In [17]:
print("실제 영화 제목 | 예측한 제목 | 리뷰")

for c in zip(y_test[:20], lr.predict(X_test_tfidf[:20]), X_test[:20]):
    print(c)
    print()

실제 영화 제목 | 예측한 제목 | 리뷰
0.65
('범죄도시', '신과함께', '오랜만에 잼나는 영화 봤습니다.  다음에 더 재미있는 영화 기대하겠습니다.')

('범죄도시', '범죄도시', '조연들이 눈에 박힌다. 간만에 집중 ㅎ')

('코코', '코코', '대감동을 선사. 인사이드 아웃을 잇는 픽사의 감동스토리. 신과함께의 멕시코판이라고나할까요??')

('신과함께', '신과함께', '돈이 안아까웠던 영화ᆞᆞ  정말 좋았다')

('신과함께', '신과함께', '역시 김용화감독이 영화는 잘 만들어요. 이제 VFX 제작 부문도 헐리우드 수준 이상입니다.')

('택시운전사', '택시운전사', '민주화를 위해 힘써주신 분들께 감사하는 마음으로 살아야겠다.')

('신과함께', '신과함께', '잠만 자다 왔음')

('신과함께', '신과함께', '오랜만에 잼있고 좋은 영화를 봤다')

('범죄도시', '신과함께', '잼남')

('범죄도시', '인피니티 워', '대박~~')

('인피니티 워', '인피니티 워', '불과 며칠전에 저스티스리그봤는데...............10점을 안 줄 수가 없다.')

('신과함께', '신과함께', '개인적으로 정말 재밌었다')

('범죄도시', '신과함께', '극장에서 본 한국영화중, 첨으로  돈  안까움')

('범죄도시', '범죄도시', '비스티보이즈  흥행은 못했지만 영화 속의 윤계상씨가 참 매력적이다 생각했는데  이번에 정말 자기 옷을 입었다는 생각이 듭니다 윤계상이라는 배우는 자상함 속에 마초성이 있다는 생각이 들었었는데 유감없이 그 숨어있던 마초적 매력이 포텐이 터진것 같아요')

('인피니티 워', '곤지암', '이 영화를 보니 디씨코믹스 영화들은 코흘리개라고 느끼게 되었다..')

('택시운전사', '신과함께', '시도는 좋았으나 결과는 참옥하다.')

('택시운전사', '신과함께', '좋은 영화입니다')

('신과함께', '신과함께', '보면서  좀  지루하기도 하고 . . . 그런데  그렇게 슬프지 않았는